## Stage 1

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from copy import deepcopy as cp
import gc; gc.enable()

In [ ]:
train_usecols = ['region', 'city', 'parent_category_name', 'category_name', 'title', 'description', 'deal_probability']
test_usecols = cp(train_usecols)
test_usecols.remove('deal_probability')

In [ ]:
train_df = pd.read_csv("data/train.csv", usecols=train_usecols)
test_df = pd.read_csv("data/test.csv", usecols=test_usecols)
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)

In [ ]:
test_df['deal_probability'] = np.zeros((test_df.shape[0],))
all_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

In [ ]:
display(train_df.head(5))
display(test_df.head(5))

In [ ]:
def print_contains_info(df, col1, col2):
    str1s = df[col1].values
    str2s = df[col2].values
    
    contains_count = 0
    for i in range(len(str1s)):
        str1 = str(str1s[i])
        str2 = str(str2s[i])
        str1 = str1.split(" ")
        str2 = str2.split(" ")
        
        for s in str1:
            if s in str2:
                contains_count += 1
                break
                
    print('{} in {} contains counts:\n'.format(col1, col2), contains_count)

In [ ]:
print_contains_info(all_df, 'region', 'title')
print_contains_info(all_df, 'city', 'title')
print_contains_info(all_df, 'parent_category_name', 'title')
print_contains_info(all_df, 'category_name', 'title')
print_contains_info(all_df, 'region', 'description')
print_contains_info(all_df, 'city', 'description')
print_contains_info(all_df, 'parent_category_name', 'description')
print_contains_info(all_df, 'category_name', 'description')
print_contains_info(all_df, 'title', 'description')

In [ ]:
def get_contains_feature(df, col1, col2):
    print('processing ', col1, col2)
    res_df = pd.DataFrame()
    str1s = df[col1].values
    str2s = df[col2].values
    
    is_contains = []
    contains_counts = []
    for i in range(len(str1s)):
        str1 = str(str1s[i])
        str2 = str(str2s[i])
        str1 = str1.split(" ")
        str2 = str2.split(" ")
        
        contains_count = 0
        for s in str1:
            if s in str2:
                contains_count += 1
                
        is_contains.append(1 if contains_count > 0 else 0)
        contains_counts.append(contains_count)
        
    res_df['{}_in_{}'.format(col1,col2)] = is_contains
    res_df['{}_in_{}_counts'.format(col1,col2)] = contains_counts
    del is_contains, contains_counts; gc.collect()
    return res_df 

In [ ]:
all_df = pd.concat([all_df, get_contains_feature(all_df, 'region', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'city', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'parent_category_name', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'category_name', 'title')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'region', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'city', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'parent_category_name', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'category_name', 'description')],  axis=1)
all_df = pd.concat([all_df, get_contains_feature(all_df, 'title', 'description')],  axis=1)

In [ ]:
norm_cols = [c for c in all_df.columns if 'counts' in c]
for col in norm_cols:
    all_df[col] = (all_df[col] - all_df[col].mean())/all_df[col].std() 

In [ ]:
text_feature = all_df['region'] + ' ' + all_df['city'] + ' ' + \
                all_df['parent_category_name'] + ' ' + all_df['category_name'] + ' ' + \
                all_df['title'] + ' ' + all_df['description']

In [ ]:
all_df = all_df.drop(test_usecols, axis=1)
gc.collect()

In [ ]:
all_df.to_csv('text_other_features.csv', index=False)

In [ ]:
stopwords = ['а', 'е', 'и', 'ж', 'м', 'о', 'на', 'не', 'ни', 'об', 'но', 'он', 'мне', 'мои', 'мож', 'она', 'они', 'оно', 'мной', 'много', 'многочисленное', 'многочисленная', 'многочисленные', 'многочисленный', 'мною', 'мой', 'мог', 'могут', 'можно', 'может', 'можхо', 'мор', 'моя', 'моё', 'мочь', 'над', 'нее', 'оба', 'нам', 'нем', 'нами', 'ними', 'мимо', 'немного', 'одной', 'одного', 'менее', 'однажды', 'однако', 'меня', 'нему', 'меньше', 'ней', 'наверху', 'него', 'ниже', 'мало', 'надо', 'один', 'одиннадцать', 'одиннадцатый', 'назад', 'наиболее', 'недавно', 'миллионов', 'недалеко', 'между', 'низко', 'меля', 'нельзя', 'нибудь', 'непрерывно', 'наконец', 'никогда', 'никуда', 'нас', 'наш', 'нет', 'нею', 'неё', 'них', 'мира', 'наша', 'наше', 'наши', 'ничего', 'начала', 'нередко', 'несколько', 'обычно', 'опять', 'около', 'мы', 'ну', 'нх', 'от', 'отовсюду', 'особенно', 'нужно', 'очень', 'отсюда', 'в', 'во', 'вон', 'вниз', 'внизу', 'вокруг', 'вот', 'восемнадцать', 'восемнадцатый', 'восемь', 'восьмой', 'вверх', 'вам', 'вами', 'важное', 'важная', 'важные', 'важный', 'вдали', 'везде', 'ведь', 'вас', 'ваш', 'ваша', 'ваше', 'ваши', 'впрочем', 'весь', 'вдруг', 'вы', 'все', 'второй', 'всем', 'всеми', 'времени', 'время', 'всему', 'всего', 'всегда', 'всех', 'всею', 'всю', 'вся', 'всё', 'всюду', 'г', 'год', 'говорил', 'говорит', 'года', 'году', 'где', 'да', 'ее', 'за', 'из', 'ли', 'же', 'им', 'до', 'по', 'ими', 'под', 'иногда', 'довольно', 'именно', 'долго', 'позже', 'более', 'должно', 'пожалуйста', 'значит', 'иметь', 'больше', 'пока', 'ему', 'имя', 'пор', 'пора', 'потом', 'потому', 'после', 'почему', 'почти', 'посреди', 'ей', 'два', 'две', 'двенадцать', 'двенадцатый', 'двадцать', 'двадцатый', 'двух', 'его', 'дел', 'или', 'без', 'день', 'занят', 'занята', 'занято', 'заняты', 'действительно', 'давно', 'девятнадцать', 'девятнадцатый', 'девять', 'девятый', 'даже', 'алло', 'жизнь', 'далеко', 'близко', 'здесь', 'дальше', 'для', 'лет', 'зато', 'даром', 'первый', 'перед', 'затем', 'зачем', 'лишь', 'десять', 'десятый', 'ею', 'её', 'их', 'бы', 'еще', 'при', 'был', 'про', 'процентов', 'против', 'просто', 'бывает', 'бывь', 'если', 'люди', 'была', 'были', 'было', 'будем', 'будет', 'будете', 'будешь', 'прекрасно', 'буду', 'будь', 'будто', 'будут', 'ещё', 'пятнадцать', 'пятнадцатый', 'друго', 'другое', 'другой', 'другие', 'другая', 'других', 'есть', 'пять', 'быть', 'лучше', 'пятый', 'к', 'ком', 'конечно', 'кому', 'кого', 'когда', 'которой', 'которого', 'которая', 'которые', 'который', 'которых', 'кем', 'каждое', 'каждая', 'каждые', 'каждый', 'кажется', 'как', 'какой', 'какая', 'кто', 'кроме', 'куда', 'кругом', 'с', 'т', 'у', 'я', 'та', 'те', 'уж', 'со', 'то', 'том', 'снова', 'тому', 'совсем', 'того', 'тогда', 'тоже', 'собой', 'тобой', 'собою', 'тобою', 'сначала', 'только', 'уметь', 'тот', 'тою', 'хорошо', 'хотеть', 'хочешь', 'хоть', 'хотя', 'свое', 'свои', 'твой', 'своей', 'своего', 'своих', 'свою', 'твоя', 'твоё', 'раз', 'уже', 'сам', 'там', 'тем', 'чем', 'сама', 'сами', 'теми', 'само', 'рано', 'самом', 'самому', 'самой', 'самого', 'семнадцать', 'семнадцатый', 'самим', 'самими', 'самих', 'саму', 'семь', 'чему', 'раньше', 'сейчас', 'чего', 'сегодня', 'себе', 'тебе', 'сеаой', 'человек', 'разве', 'теперь', 'себя', 'тебя', 'седьмой', 'спасибо', 'слишком', 'так', 'такое', 'такой', 'такие', 'также', 'такая', 'сих', 'тех', 'чаще', 'четвертый', 'через', 'часто', 'шестой', 'шестнадцать', 'шестнадцатый', 'шесть', 'четыре', 'четырнадцать', 'четырнадцатый', 'сколько', 'сказал', 'сказала', 'сказать', 'ту', 'ты', 'три', 'эта', 'эти', 'что', 'это', 'чтоб', 'этом', 'этому', 'этой', 'этого', 'чтобы', 'этот', 'стал', 'туда', 'этим', 'этими', 'рядом', 'тринадцать', 'тринадцатый', 'этих', 'третий', 'тут', 'эту', 'суть', 'чуть', 'тысяч']

In [ ]:
def clean_text(txt):
    words = str(txt).split(str=" ./")
    words = [wrd for wrd in words if wrd not in stopwords]
    words = [wrd for wrd in words if len(wrd) > 1]
    txt = " ".join(words)
    return txt

In [ ]:
text_feature = text_feature.apply(lambda x: clean_text(x))

In [ ]:
gc.collect()

In [ ]:
import pickle

In [ ]:
with open('text_feature.pickle', 'wb') as handle:
    pickle.dump(text_feature, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Stage 2

In [1]:
import pickle
import numpy as np
import gc; gc.enable()
with open('text_feature.pickle', 'rb') as handle:
    text_feature = pickle.load(handle)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

In [3]:
hash_char_wb_features = HashingVectorizer(ngram_range=(2, 3), 
                                          analyzer='char_wb',
                                          n_features=2**18,
                                          norm='l2', 
                                          dtype=np.float32).fit_transform(text_feature.values)
print(hash_char_wb_features.shape)

(2011862, 262144)


In [4]:
with open('hash_char_wb_features.pickle', 'wb') as handle:
    pickle.dump(hash_char_wb_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
hash_char_wb_features = (hash_char_wb_features > 0).astype(int)
with open('hash_char_wb_bin_features.pickle', 'wb') as handle:
    pickle.dump(hash_char_wb_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
del hash_char_wb_features; gc.collect()

218

## Stage 3: Test with fm_ftrl model

In [7]:
import pickle
import pandas as pd
import numpy as np
import gc; gc.enable()
from sklearn.decomposition import TruncatedSVD

In [8]:
with open('hash_char_wb_features.pickle', 'rb') as handle:
    hash_char_wb_features = pickle.load(handle)
    
hash_char_wb_features = hash_char_wb_features.astype(np.float64)
gc.collect()

0

In [9]:
train_df = pd.read_csv("data/train.csv", usecols=['deal_probability'])
train_len = train_df.shape[0]
train_y = train_df['deal_probability'].values
del train_df
print(train_len)

1503424


In [10]:
hash_char_wb_train_features = hash_char_wb_features[:train_len, :] 
hash_char_wb_test_features = hash_char_wb_features[train_len:, :] 

In [16]:
del hash_char_wb_features; gc.collect()

306

In [17]:
hash_char_wb_train_features.shape

(1503424, 262144)

In [18]:
hash_char_wb_test_features.shape

(508438, 262144)

In [11]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV, ParameterGrid, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from copy import deepcopy as cp

In [12]:
def clip_rmse(ground_truth, predictions):
    predictions = np.clip(predictions, 0., 1.)
    return mean_squared_error(ground_truth, predictions)**.5

clip_rmse_scorer = make_scorer(clip_rmse, greater_is_better=False)

In [13]:
kfold = KFold(n_splits=3, random_state=719)

In [14]:
from wordbatch.models import FM_FTRL

In [15]:
def simple_train_test_eval(default_params, X, y, params):
    tr_X, val_X, tr_y, val_y = train_test_split(X, y, test_size=0.3, random_state=719)
    
    min_score = None
    best_param = None
    
    for param in list(ParameterGrid(params)):
        use_params = cp(default_params)
        use_params.update(param)
        print('Fitting params:\n', use_params)
        md = FM_FTRL(**use_params)
        md.fit(tr_X, tr_y)
        score = clip_rmse(val_y, md.predict(val_X))
        print(param, score)
        
        if min_score is None or score < min_score:
            best_param = param
            min_score = score
            
    print('Best param:', best_param, '\nscore:', min_score)

In [22]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.0001,
    'L2': 0.1,
    'D': hash_char_wb_train_features.shape[1],
    'D_fm': 300,
    'iters': 40,
    'seed': 719,
    'threads': 4,
    'verbose': 0
}

try_params = {
    'L1': [0.0001],
    'L2': [0.1]
}
simple_train_test_eval(fmftrl_default_params, hash_char_wb_train_features, train_y, try_params)

Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.0001, 'L2': 0.1, 'D': 262144, 'D_fm': 300, 'iters': 40, 'seed': 719, 'threads': 4, 'verbose': 0}
{'L1': 0.0001, 'L2': 0.1} 0.233972783022
Best param: {'L1': 0.0001, 'L2': 0.1} 
score: 0.233972783022
